# Sandbox
For playing around during development.

In [1]:
%pylab inline
import pysd
print pysd.__version__
print pysd.__file__
import pandas as pd

Populating the interactive namespace from numpy and matplotlib
0.3.2
pysd/__init__.pyc


# Testing various subscript Options


In [26]:
from __future__ import division 
import numpy as np 
from pysd import functions
from scipy.integrate import odeint
import itertools

### option 1: unnamed array

In [27]:
class MinModel(object):
    ##########  boilerplate stuff from the existing pysd #########
    def __init__(self):
        self._stocknames = [name[:-5] for name in dir(self) if name[-5:] == '_init']
        self._stocknames.sort() #inplace
        self._dfuncs = [getattr(self, 'd%s_dt'%name) for name in self._stocknames]
        self.state = dict(zip(self._stocknames, [None]*len(self._stocknames)))
        self.reset_state()
        self.functions = functions.Functions(self)

    def reset_state(self):
        """Sets the model state to the state described in the model file. """
        self.t = self.initial_time() #set the initial time
        retry_flag = False
        for key in self.state.keys():
            try:
                self.state[key] = eval('self.'+key+'_init()') #set the initial state
            except TypeError:
                retry_flag = True
        if retry_flag:
            self.reset_state() #potential for infinite loop!

    ########### Stuff we have to modify to make subscripts work #########
    def d_dt(self, state_vector, t):
        """The primary purpose of this function is to interact with the integrator.
        It takes a state vector, sets the state of the system based on that vector,
        and returns a derivative of the state vector
        """        
        self.set_state(state_vector)
        self.t = t
        
        derivative_vector = []
        for func in self._dfuncs:
            derivative_vector += list(func())
            
        return derivative_vector

    def set_state(self, state_vector):
        i = 0
        for key in self._stocknames:
            if isinstance(self.state[key], np.ndarray):
                size = self.state[key].size
                elements = state_vector[i:i+size]
                shape = self.state[key].shape
                self.state[key] = np.array(elements).reshape(shape)
                i += size
            else:
                self.state[key] = state_vector[i]
                i += 1

        
    def get_state(self):
        #if we keep this, we should make it fully a list comprehension
        state_vector = []
        for item in [self.state[key] for key in self._stocknames]:
            if isinstance(item, np.ndarray):
                state_vector += list(item.flatten())
            else:
                state_vector += list(item)
        return state_vector
    
    
    ######### model specific components (that go in the model file)
    suba_list = ['suba1', 'suba2', 'suba3']
    subb_list = ['suba2', 'subb2']
    suba_index = dict(zip(suba_list, range(len(suba_list))))
    subb_index = dict(zip(subb_list, range(len(subb_list))))
    
    def stock(self, suba, subb):
        return self.state['stock'][self.suba_index[suba]][self.subb_index[subb]] 
    
    def stock_init(self):
        return np.array([[1,1],[1,1],[1,1]])
    
    def dstock_dt(self):
        return [self.flow(suba, subb) for suba, subb in itertools.product(self.suba_list, self.subb_list)]
    
    def constant(self, suba, subb):
        return self.constant.values[self.suba_index[suba]][self.subb_index[subb]]
    constant.values = np.array([[1,2],[3,4],[5,6]])
    
    def flow(self, suba, subb):
        return self.constant(suba, subb) * self.stock(suba, subb)
    
    def initial_time(self):
        return 0

In [28]:
a = MinModel()

In [29]:
%%timeit
a.reset_state()
odeint(a.d_dt, a.get_state(), range(10))

100 loops, best of 3: 16.4 ms per loop


In [34]:
%%prun
odeint(a.d_dt, a.get_state(), range(10))

### option 2: xray's `DataArray`

In [30]:
from xray import DataArray

class xMinModel(object):
    ##########  boilerplate stuff from the existing pysd #########
    def __init__(self):
        self._stocknames = [name[:-5] for name in dir(self) if name[-5:] == '_init']
        self._stocknames.sort() #inplace
        self._dfuncs = [getattr(self, 'd%s_dt'%name) for name in self._stocknames]
        self.state = dict(zip(self._stocknames, [None]*len(self._stocknames)))
        self.reset_state()
        self.functions = functions.Functions(self)

    def reset_state(self):
        """Sets the model state to the state described in the model file. """
        self.t = self.initial_time() #set the initial time
        retry_flag = False
        for key in self.state.keys():
            try:
                self.state[key] = eval('self.'+key+'_init()') #set the initial state
            except TypeError:
                retry_flag = True
        if retry_flag:
            self.reset_state() #potential for infinite loop!

    ########### Stuff we have to modify to make subscripts work #########
    def d_dt(self, state_vector, t):
        """The primary purpose of this function is to interact with the integrator.
        It takes a state vector, sets the state of the system based on that vector,
        and returns a derivative of the state vector
        """        
        self.set_state(state_vector)
        self.t = t
        
        derivative_vector = []
        for func in self._dfuncs:
            derivative_vector += list(func())
            
        return derivative_vector

    def set_state(self, state_vector):
        i = 0
        for key in self._stocknames:
            if isinstance(self.state[key], DataArray):
                shape = self.state[key].shape
                size = self.state[key].size
                self.state[key].loc[:,:] = np.array(state_vector[i:i+size]).reshape(shape)
                i += size
            else:
                self.state[key] = state_vector[i]
                i += 1
        
    def get_state(self):
        #if we keep this, we should make it fully a list comprehension
        state_vector = []
        for item in [self.state[key] for key in self._stocknames]:
            if isinstance(item, DataArray):
                state_vector += list(item.values.flatten())
            else:
                state_vector += list(item)
        return state_vector
    
    
    ######### model specific components (that go in the model file)
    dim_dict = {'suba': ['subb1', 'subb2'],
                'subb': ['suba1', 'suba2', 'suba3']}
    
    def stock(self, suba, subb):
        return self.state['stock'].loc[suba, subb].values
    
    def stock_init(self):
        return DataArray([[1,1],[1,1],[1,1]], self.dim_dict)
    
    def dstock_dt(self):
        return [self.flow(suba, subb) for suba, subb in itertools.product(*self.dim_dict.values())]
    
    def constant(self, suba, subb):
        #values = DataArray([[1,2],[3,4],[5,6]], self.dim_dict)
        return self.constant.values.loc[suba, subb].values
    constant.values = DataArray([[1,2],[3,4],[5,6]], dim_dict)
    
    def flow(self, suba, subb):
        return self.constant(suba, subb) * self.stock(suba, subb)
    
    def initial_time(self):
        return 0

In [32]:
a = xMinModel()

In [33]:
%%timeit
a.reset_state()
odeint(a.d_dt, a.get_state(), range(10))

Excess work done on this call (perhaps wrong Dfun type).
Run with full_output = 1 to get quantitative information.
Excess work done on this call (perhaps wrong Dfun type).
Run with full_output = 1 to get quantitative information.
Excess work done on this call (perhaps wrong Dfun type).
Run with full_output = 1 to get quantitative information.
Excess work done on this call (perhaps wrong Dfun type).
Run with full_output = 1 to get quantitative information.
1 loops, best of 3: 3.41 s per loop


In [26]:
%%prun
odeint(a.d_dt, a.get_state(), range(10))

Excess work done on this call (perhaps wrong Dfun type).
Run with full_output = 1 to get quantitative information.
 

# Option 3

Instead of calling the functions each time for each value of the subscript, can we call a function once, and get all of the values? Then we're doing array mathematics instead of a bunch of one-off multiplication.

This is nice, and faster, but i'm not sure how we would scale it to deal with elements with different functions.

In [34]:
class fMinModel(object):
    ##########  boilerplate stuff from the existing pysd #########
    def __init__(self):
        self._stocknames = [name[:-5] for name in dir(self) if name[-5:] == '_init']
        self._stocknames.sort() #inplace
        self._dfuncs = [getattr(self, 'd%s_dt'%name) for name in self._stocknames]
        self.state = dict(zip(self._stocknames, [None]*len(self._stocknames)))
        self.reset_state()
        self.functions = functions.Functions(self)

    def reset_state(self):
        """Sets the model state to the state described in the model file. """
        self.t = self.initial_time() #set the initial time
        retry_flag = False
        for key in self.state.keys():
            try:
                self.state[key] = eval('self.'+key+'_init()') #set the initial state
            except TypeError:
                retry_flag = True
        if retry_flag:
            self.reset_state() #potential for infinite loop!

    ########### Stuff we have to modify to make subscripts work #########
    def d_dt(self, state_vector, t):
        """The primary purpose of this function is to interact with the integrator.
        It takes a state vector, sets the state of the system based on that vector,
        and returns a derivative of the state vector
        """        
        self.set_state(state_vector)
        self.t = t
        
        derivative_vector = []
        for func in self._dfuncs:
            res = func()
            if isinstance(res, np.ndarray):
                res = res.flatten()
            derivative_vector += list(res)
            
        return derivative_vector

    def set_state(self, state_vector):
        i = 0
        for key in self._stocknames:
            
            if isinstance(self.state[key], np.ndarray):
                size = self.state[key].size
                elements = state_vector[i:i+size]
                shape = self.state[key].shape
                self.state[key] = np.array(elements).reshape(shape)
                i += size
            else:
                self.state[key] = state_vector[i]
                i += 1

        
    def get_state(self):
        #if we keep this, we should make it fully a list comprehension
        state_vector = []
        for item in [self.state[key] for key in self._stocknames]:
            if isinstance(item, np.ndarray):
                state_vector += list(item.flatten())
            else:
                state_vector += list(item)
        return state_vector
    
    
    ######### model specific components (that go in the model file)
    
    def stock(self):
        return self.state['stock']
    
    def stock_init(self):
        return np.array([[1,1],[1,1],[1,1]])
    
    def dstock_dt(self):
        return self.flow()
    
    def constant(self):
        return np.array([[1,2],[3,4],[5,6]])
    
    def constant2(self):
        return 6
    
    def flow(self):
        return self.constant() * self.stock()
    
    def initial_time(self):
        return 0

In [35]:
a = fMinModel()

In [36]:
%%timeit
a.reset_state()
odeint(a.d_dt, a.get_state(), range(100))

10 loops, best of 3: 96.1 ms per loop


# Option 5 


In [65]:
class eMinModel(object):
    def __init__(self):
        self._stocknames = [name[:-5] for name in dir(self) if name[-5:] == '_init']
        self._stocknames.sort()
        self._dfuncs = {name: getattr(self, 'd%s_dt' % name) for name in self._stocknames}
        self.state = dict(zip(self._stocknames, [None]*len(self._stocknames)))
        self.reset_state()
        self.functions = functions.Functions(self)

    def reset_state(self):
        self.t = self.initial_time() #set the initial time
        retry_flag = False
        for key in self.state.keys():
            try:
                self.state[key] = eval('self.'+key+'_init()') #set the initial state
            except TypeError:
                retry_flag = True
        if retry_flag:
            self.reset_state() #potential for infinite loop!
    
    def step(self, dt):
        newstate = {}
        self.state = {key:self._dfuncs[key]() * dt + self.state[key] for key in self._stocknames}
        self.t = self.t+dt
        return self.state
        
    def integrate(self, timesteps):
        outputs = range(len(timesteps))
        return [self.step(t2-self.t) for t2 in timesteps]
#         for i, t2 in enumerate(timesteps):
#             outputs[i] = self.step(t2-self.t)
#         return outputs
    
    ######### model specific components (that go in the model file)
    
    def stock(self):
        return self.state['stock']
    
    def stock_init(self):
        return np.array([[1,1],[1,1],[1,1]])
    
    def dstock_dt(self):
        return self.flow()
    
    def constant(self):
        return np.array([[1,2],[3,4],[5,6]])
    
    def constant2(self):
        return 6
    
    def flow(self):
        return self.constant() * self.stock()
    
    def initial_time(self):
        return 0

In [66]:
a = eMinModel()

In [67]:
a.state

{'stock': array([[1, 1],
        [1, 1],
        [1, 1]])}

In [71]:
%%timeit
a.reset_state()
a.integrate(np.arange(0,10,.1))

1000 loops, best of 3: 797 µs per loop
